In [2]:
import torch
import transformers
from peft import (
    LoraConfig, 
    get_peft_model,
    TaskType, 
)
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    SchedulerType,
    get_scheduler,
    BitsAndBytesConfig,
    set_seed,
    DataCollatorForSeq2Seq,
)

In [10]:
model = AutoModelForSeq2SeqLM.from_pretrained(
    "t5-small",
    torch_dtype=torch.bfloat16,
    device_map={"": 1},
)

In [12]:
peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    inference_mode=False,
    r=16,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="all",
)
model = get_peft_model(model, peft_config)

In [22]:
model.model.base_model.model.encoder.block[0].layer[0].SelfAttention.q

Linear(
  in_features=512, out_features=512, bias=False
  (lora_dropout): ModuleDict(
    (default): Dropout(p=0.1, inplace=False)
  )
  (lora_A): ModuleDict(
    (default): Linear(in_features=512, out_features=16, bias=False)
  )
  (lora_B): ModuleDict(
    (default): Linear(in_features=16, out_features=512, bias=False)
  )
  (lora_embedding_A): ParameterDict()
  (lora_embedding_B): ParameterDict()
)

In [23]:
model.model.base_model.model.encoder.block[0].layer[0].SelfAttention.q.weight

Parameter containing:
tensor([[-0.0162, -0.0737,  0.0042,  ...,  0.0271, -0.0366, -0.1191],
        [-0.0371, -0.0227,  0.0491,  ..., -0.0427, -0.0486,  0.0669],
        [ 0.0540, -0.0566,  0.0054,  ..., -0.0339, -0.0173, -0.0430],
        ...,
        [-0.0908,  0.0104, -0.1104,  ..., -0.0070, -0.1001,  0.0405],
        [ 0.0017,  0.0157, -0.0427,  ..., -0.0962,  0.0300,  0.0015],
        [-0.1069,  0.0002, -0.0029,  ...,  0.0530, -0.0432, -0.0522]],
       device='cuda:1', dtype=torch.bfloat16)

In [29]:
model.model.base_model.model.encoder.block[0].layer[0].SelfAttention.q.lora_A.default.weight.dtype

torch.float32